In [32]:
import pandas as pd
import spacy
import spacy.cli
spacy.cli.download("en_core_web_sm")
spacy.cli.download("en_core_web_lg")
from pathlib import Path

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [10]:
#loading model
sp_sm = spacy.load('en_core_web_sm')

In [11]:
#creating function to identify entities
def spacy_sm_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in sp_sm(document).ents}

In [12]:
yelp_pos_filtered = pd.read_csv("/Users/Anusha/PycharmProjects/yelp-app/data/yelp_pos_filtered.csv")

In [13]:
yelp_pos_filtered.head()

,Unnamed: 0.1,Unnamed: 0,business_id,name,state,stars,categories,text,funny,cool,useful,pos_sentiment
0,2,2,6OTCxq9JQDRNwakXI9k3HQ,Thai Basil,FL,4.0,"Restaurants, Thai","IGNORANT STAFF SEEMED NICE\nRepeat - I said ""N...",2,2,2,0.964604
1,4,4,-xaY1TlMf20Ol9QLcDSU6Q,French Market,LA,3.5,"Farmers Market, Specialty Food, Shopping Cente...",The French Market is an old outdoor marketplac...,2,1,3,0.885489
2,21,21,EcjHDndSeQuebMMJhCGC5g,Yangming,PA,3.5,"Chinese, Nightlife, Restaurants, Bars",Not sure how it has had the reputation it trum...,1,0,7,0.987738
3,22,22,5u_eZ2_GYsPtLuRwAajgzg,Birchrunville Store Cafe,PA,4.0,"French, Restaurants, Italian, Cafes",Beware of temperamental French chef!!! Wow. Do...,0,0,7,0.969046
4,27,27,x_teiEdMOErcpAmqisUD5g,HDTV City,AZ,4.0,"Home Services, Home Theatre Installation, Prof...",Two stars is for the service. Other than that ...,1,0,10,0.852800


In [15]:
#testing model
example_text = yelp_pos_filtered['text'][0]
print(example_text)

IGNORANT STAFF SEEMED NICE
Repeat - I said "No nuts please. I'm allergic to nuts" a couple times and had it repeated to me on the phone. But shortly after tasting all of my take-out from this place my stomach was swelling and I was huddled near the toilet for almost three hours. Mind you I can have 'peanut traces' and peanut oil and be totally fine (it takes a lot to spark my reaction)- so this was an obvious disregard of my request. Because of this ignorance my reviews are a bit biased but imagine if I was one of those people who go into a breathing shut-down upon ingesting nuts? And if they don't communicate to each other about this, what else don't they communicate on?

WELL WHAT ABOUT THE FOOD
Before my body started rejecting the food, I had tasted enough dishes. The Toasted Tortilla appetizer was unique. It seemed similar to a Philippine Lumpia with its crispiness and accompanying sweet sauce except it was quesadilla shaped. Our Chicken Panang Curry was more watery than I'd like a

In [16]:
spacy_sm_ner(example_text)
#model is identifying food as 'PERSON'

{('Chicken', 'PERSON'),
 ('Chicken Panang Curry', 'PERSON'),
 ('Lumpia', 'PERSON'),
 ('Pad Thai', 'PERSON'),
 ('Philippine', 'NORP'),
 ('Sugar', 'GPE'),
 ('almost three hours', 'TIME')}

In [18]:
sp_lg = spacy.load('en_core_web_lg')

In [22]:
#retry using large model

def spacy_lg_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in sp_lg(document).ents}

In [23]:
spacy_lg_ner(example_text) #better, but some food is still classified as 'PERSON'

{('Chicken', 'NORP'),
 ('Lumpia', 'PRODUCT'),
 ('Pad Thai', 'GPE'),
 ('Panang Curry', 'PERSON'),
 ('Philippine', 'NORP'),
 ('Sugar', 'PRODUCT'),
 ('almost three hours', 'TIME')}

In [ ]:
yelp_pos_filtered['NER'] = ''
for i in range(len(yelp_pos_filtered)):
    yelp_pos_filtered['NER'][i] = spacy_lg_ner(yelp_pos_filtered['text'][i])

In [30]:
yelp_pos_filtered.head()

,Unnamed: 0.1,Unnamed: 0,business_id,name,state,stars,categories,text,funny,cool,useful,pos_sentiment,NER
0,2,2,6OTCxq9JQDRNwakXI9k3HQ,Thai Basil,FL,4.0,"Restaurants, Thai","IGNORANT STAFF SEEMED NICE\nRepeat - I said ""N...",2,2,2,0.964604,"{(Pad Thai, GPE), (Panang Curry, PERSON), (alm..."
1,4,4,-xaY1TlMf20Ol9QLcDSU6Q,French Market,LA,3.5,"Farmers Market, Specialty Food, Shopping Cente...",The French Market is an old outdoor marketplac...,2,1,3,0.885489,"{(the French Quarter, EVENT), (NOLA, PERSON), ..."
2,21,21,EcjHDndSeQuebMMJhCGC5g,Yangming,PA,3.5,"Chinese, Nightlife, Restaurants, Bars",Not sure how it has had the reputation it trum...,1,0,7,0.987738,"{(years, DATE), (night, TIME), (the previous d..."
3,22,22,5u_eZ2_GYsPtLuRwAajgzg,Birchrunville Store Cafe,PA,4.0,"French, Restaurants, Italian, Cafes",Beware of temperamental French chef!!! Wow. Do...,0,0,7,0.969046,"{(Three years ago, DATE), (4, CARDINAL), (the ..."
4,27,27,x_teiEdMOErcpAmqisUD5g,HDTV City,AZ,4.0,"Home Services, Home Theatre Installation, Prof...",Two stars is for the service. Other than that ...,1,0,10,0.852800,"{(kidding???After, PERSON), (Two, CARDINAL), (..."


In [31]:
yelp_pos_filtered['NER'].isnull().sum().sum()

0

In [33]:
filepath = Path('/Users/Anusha/PycharmProjects/yelp-app/data/yelp_pos_filtered_ner.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
yelp_pos_filtered.to_csv(filepath)